<a href="https://colab.research.google.com/github/ApoorvAkash/Coursera_Capstone/blob/main/Neighborhoods_Toronto_BeautifulSoup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Segmenting and Clustering Neighbourhoods in Toronto

### Importing Libraries

In [1]:
# Geocoder Python Package
!pip install opencage

In [2]:
import pandas as pd

import requests 
from bs4 import BeautifulSoup

import numpy as np

from opencage.geocoder import OpenCageGeocode
key = '63a8e0494a2f4af7b0fd2b1b6634006b'


### Create BeautifulSoup Object

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'html.parser')
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


### Create DF from BeautifulSoup Object

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### EDA

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M4K,North York,Davisville
freq,1,24,1


In [7]:
df.isna().value_counts()

PostalCode  Borough  Neighborhood
False       False    False           103
dtype: int64

In [8]:
df.shape

(103, 3)

### Saving Dataset

In [14]:
df.to_csv('neighborhoods_toronto_data.csv')

### Geocoding the Neighborhoods

In [9]:
# loop until you get the coordinates
postal_codes = df['PostalCode']
lat_lng_array = []

geocoder = OpenCageGeocode(key) 

for postal_code in postal_codes:
  result = geocoder.geocode('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = result[0]['geometry']
  lat_lng_array.append({'PostalCode': postal_code, 'latitude': lat_lng_coords['lat'], 'longitude': lat_lng_coords['lng']})


In [10]:
lat_lng_df = pd.DataFrame(lat_lng_array)
lat_lng_df.shape

(103, 3)

In [11]:
lat_lng_df.head(5)

,PostalCode,latitude,longitude
0,M3A,43.653482,-79.383935
1,M4A,43.727600,-79.314800
2,M5A,43.655500,-79.362600
3,M6A,43.722300,-79.450400
4,M7A,43.653482,-79.383935


### Merging and Saving the Dataset

In [12]:
df_with_lat_lng = pd.merge(df, lat_lng_df, on='PostalCode')
df_with_lat_lng

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722300,-79.450400
4,M7A,Queen's Park,Ontario Provincial Government,43.653482,-79.383935
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.651800,-79.507600
99,M4Y,Downtown Toronto,Church and Wellesley,43.665600,-79.383000
100,M7Y,East Toronto Business,Enclave of M4L,43.780400,-79.250500
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632500,-79.493900
